## Purpose: Includes the GUIDs that have one or more gene variant types from the eyeGeneGenomics data, and one or more genes. 

1. Input: Data from the eyeGeneGenomics filtered on the gene variant type. 
2. Output: List of GUIDs that have all three gene variant types. 

In [ ]:
import pandas as pd # used for creating a dataframe and other methods using pandas
import requests # used to API calls
import json # used for creating json files
import getpass
from io import StringIO
import os
import datetime as dt
import time
import sys

In [ ]:
def create_folder(folder_path):
    adjusted_folder_path = folder_path
    folder_found = os.path.isdir(adjusted_folder_path)
    counter = 0
    while folder_found == True:
        counter = counter + 1
        adjusted_folder_path = folder_path + " (" + str(counter) + ")"
        folder_found = os.path.isdir(adjusted_folder_path)
    os.mkdir(adjusted_folder_path)
    return adjusted_folder_path

x=dt.datetime.now()
new_dir = os.getcwd()+"\\"+"EyeGene_"+x.strftime("%Y_%m_%d")+"_outputfiles"
created_dir = create_folder(new_dir)

Enter User Credentials

In [ ]:
## login
print("Enter your username")
username = input()

password = getpass.getpass("Enter your password")

In [ ]:
loginheaders = {
    "accept": "text/plain",
    "Content-Type": "application/x-www-form-urlencoded"
}

logindata = {"password":password,
         "username": username}

In [ ]:
response = requests.post("https://brics.nei.nih.gov/gateway/authentication/user/login", headers=loginheaders, data=logindata)
#login in check 
if response.status_code == 200:
    print("Login Successful")
    token=response.text
    print(token)
elif response.status_code != 200:
    print(response.status_code)
    print("Login not Successful. Please check username and password. If error still occurs reach out to system adminstrator. THIS CODE WILL NOT PROCEED")
    

## Enter values for query

In [ ]:
genelist = input("Enter list of genes")
genevarianttype =input("Enter gene variant variation type")

In [ ]:
genes = genelist.split(",")
genevariantlist = genevarianttype.split(",")

In [ ]:
#check
genePVs = ["benign", "likely benign","likely pathogenic", "pathogenic","uncertain significance"]

for value in genevariantlist:
    if value in genePVs:
        print(value + " is in list")
    else: 
        print(value + "   is not in list.Please reenter values before proceeding")
        break

In [ ]:
genefilter = []
for i in range(0,len(genes)): 
    dataelement={
            "dataElement": "HGNCGeneSymbl",
            "form": "eyeGENEGenomics",
            "repeatableGroup": "Genomics Information",
            "operator":"OR",
            "value":[genes[i].strip("")]}
    genefilter.append(dataelement)

In [ ]:
genefilter

In [ ]:
genomicsfilter = {
  "filter": genefilter,
  "flattened": "false",
  "formStudy": [
    {
      "form":"eyeGENEGenomics",
      "studies": ["EYEGENE-STUDY0000203"]
    }
  ]
}

In [ ]:
queryurl ="https://brics.nei.nih.gov/gateway/query-api/data/csv"

headers = {
    "accept": "application/csv",
    "Content-type": "application/json",
    "Authorization":"Bearer " + token }

In [ ]:
%time query = requests.post(queryurl,headers=headers,json=genomicsfilter)
query

In [ ]:
print(f"Response: {query}")
print("Data received: " + query.headers["Content-Disposition"][21:96])

In [ ]:
query.headers

In [ ]:
dataset = query.text

In [ ]:
texttodf = StringIO(dataset)    
nei_data = pd.read_csv(texttodf, sep=",")
print(len(nei_data))
nei_data.head()

In [ ]:
inputdata =nei_data

In [ ]:
inputdata["eyeGENEGenomics.Genomics Information.GeneVariantInterpretTyp"]=inputdata["eyeGENEGenomics.Genomics Information.GeneVariantInterpretTyp"].str.lower()

In [ ]:
genevariantlist

In [ ]:
datawgenevariant = inputdata[inputdata["eyeGENEGenomics.Genomics Information.GeneVariantInterpretTyp"].isin(genevariantlist)]
print(len(datawgenevariant))

In [ ]:
name = {}
for gene in genes:
    name[gene] = pd.DataFrame()
    name[gene]=datawgenevariant[datawgenevariant["eyeGENEGenomics.Genomics Information.HGNCGeneSymbl"]==gene]
    print(f'There are {len(name[gene])} rows of data for name[{gene}].')
print(f'There are {len(name)} dataframes created.')
   

In [ ]:
dfs = []
for i in range(0,len(genes)):
    dfs.append(name[genes[i]])
    print(f'name{[genes[i]]}')


In [ ]:
from functools import partial, reduce

merge = partial(pd.merge, on=["eyeGENEGenomics.Main.GUID"], how='inner')
results = reduce(merge, dfs)["eyeGENEGenomics.Main.GUID"]
uniqueguids = results.unique()

In [ ]:
updated = datawgenevariant[datawgenevariant["eyeGENEGenomics.Main.GUID"].isin(uniqueguids)]

In [ ]:
updated.head()

In [ ]:
guidlist = []
genelist = []
medicalcond= []
for guid in uniqueguids:
    filtereddata = updated[updated["eyeGENEGenomics.Main.GUID"]==str(guid)][["eyeGENEGenomics.Genomics Information.HGNCGeneSymbl","eyeGENEGenomics.Main.MedicalCondNEIEnrollTyp"]]
    genelist.append(filtereddata["eyeGENEGenomics.Genomics Information.HGNCGeneSymbl"].unique().tolist())
    medicalcond.append(filtereddata["eyeGENEGenomics.Main.MedicalCondNEIEnrollTyp"].unique().tolist())
    guidlist.append(guid)

In [ ]:
glist = pd.Series(guidlist)
genelist2 = pd.Series(genelist)
medlist = pd.Series(medicalcond)

In [ ]:
ds = pd.concat([glist,genelist2,medlist],axis=1,ignore_index=True)

In [ ]:
ds.rename(columns={0:"GUID",1:"HGNCGeneSymbl",2:"MedicalCondNEIEnrollTyp"},inplace=True)

In [ ]:
print("The number of rows from query tool: " + str(len(nei_data)))
print("________________________________________________________")

print("The number of unique GUIDs that have gene variant type" + str(genevariantlist) + ":" + str(len(updated["eyeGENEGenomics.Main.GUID"].unique())))

print("________________________________________________________")

    
print(f"The filtered data for the GUIDs with gene variant type {genevariantlist} and {genes} : {len(updated)}")

print("________________________________________________________")

print("The files have been created in your folder  " + str(created_dir))
